## Chapter 4: Training Models
General overview of models and how to train them. Code based on the book, chapter IV "Training Models".

Modified to experiment with code

In [2]:
import numpy as np

In [3]:
# Generate linear data
np.random.seed(42)
m = 100
X = 2 * np.random.rand(m, 1)
y = 4 + 3 * X + np.random.randn(m, 1)

## Normal equation

![MSE](MSE.png "MSE")  
MSE  
  
![Vectorized MSE](Vectorized.png "Vectorized MSE")  
Vectorized MSE  
  
![Derivative of MSE](Derivative.png "Derivative of MSE")  
Derivative  
  
![Find min](Minimize.png "Minimize Derivative")  
Minimize derivative  
  
![Solving](Solving.png "Solving derivative")  
Solving  
  
![Normal Equation](NormalEquation.png "Result: Normal Equation")  
Solution/Normal Equation  

In [4]:
from sklearn.preprocessing import add_dummy_feature

X_b = add_dummy_feature(X) # add x0 = 1 to each instance
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)
theta_best

array([[4.21509616],
       [2.77011339]])

In [5]:
# Or more ML-like implementation
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
lin_reg.intercept_, lin_reg.coef_

(array([4.21509616]), array([[2.77011339]]))

## Gradient Descent

Gradient Descent is more flexible. Normal equation is computationally complex (O(n<sup>2.4</sup>) - O(n<sup>3</sup>)) and might not exist for all problems (i.e. non-linear relations)

![Find min](Minimize.png "Minimize Derivative")  

In [6]:
# Batch Gradient Descent
eta = 0.1 # learning rate
n_iterations = 1000

np.random.seed(42)
theta = np.random.randn(2,1) # randomly initialized model

for epoch in range(n_iterations):
    gradients = 2/len(X_b) * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients
    
theta

array([[4.21509616],
       [2.77011339]])

In [7]:
# Stochastic Gradient Descent
n_iterations = 50
t0, t1 = 5, 50 # learning schedule hyperparameters

# Define a learning schedule to decrease the learning rate over time
def learning_schedule(t):
    return t0 / (t + t1)

np.random.seed(42)
theta = np.random.randn(2,1) # randomly initialized model

for epoch in range(n_iterations):
    for i in range(len(X_b)):
        random_index = np.random.randint(len(X_b))
        xi = X_b[random_index:random_index+1] # pick a random instance
        yi = y[random_index:random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * len(X_b) + i)
        theta = theta - eta * gradients

theta

array([[4.21076011],
       [2.74856079]])

In [8]:
# More ML-like implementation
from sklearn.linear_model import SGDRegressor

sgd_reg = SGDRegressor(max_iter=1000,tol=1e-5, penalty=None, eta0=0.01,
                       n_iter_no_change=100, random_state=42)
# tol: tolerance for stopping criteria
# penalty: regularization term (None for no regularization)
# eta0: initial learning rate
# n_iter_no_change: number of iterations with no improvement before stopping

sgd_reg.fit(X, y.ravel()) # ravel() flattens y into a 1D array

sgd_reg.intercept_, sgd_reg.coef_

(array([4.21278812]), array([2.77270267]))

## Polynomial Regression

In [9]:
# Generate nonlinear data
np.random.seed(42)
m = 100
X = 6 * np.random.rand(m, 1) - 3 # random values between -3 and 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1) # quadratic function

In [10]:
from sklearn.preprocessing import PolynomialFeatures

poly_features = PolynomialFeatures(degree=2, include_bias=False)
# Transform training data to a 2nd degree polynomial adding second-degree polynomial (but no bias term (x^0))
X_poly = poly_features.fit_transform(X)
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

(array([1.78134581]), array([[0.93366893, 0.56456263]]))

## Regularization

### Ridge Regression / L2 Regularization
Addition of regularization term to MSE   
!["Regularization Term"](Rigeterm.png)  
  
This equals the square of the L2 Norm  
!["L2 Norm"](L2Norm.png)  
L2 Norm  

  
This leads to full MSE as  
!["MSE with L2 Regularization"](rigeregression.png)  
  
Or (easier/more famous) as:  
!["MSE with L2 Regularization"](rige2.png)  
  
  
Again, this could be solved in closed-form (via Cholesky matrix factorization) or Gradient Descent.

In [11]:
sgd_reg = SGDRegressor(penalty='l2', alpha=0.1 / m, tol=None,
                          max_iter=1000, eta0=0.01, random_state=42)
# penalty: Regularization term (l2 = Ridge, l1 = Lasso)
# Because l2 does not divide the regularization term by m, it has to be added

sgd_reg.fit(X_poly, y.ravel()) # ravel() flattens y into a 1D array
sgd_reg.intercept_, sgd_reg.coef_

(array([1.78211323]), array([0.93420757, 0.56518136]))

### Lasso Regression / L1 Regularization

Lasso Regression adds the L1 Norm to the MSE (but not squared like Ridge Regression)  
!["L1 Norm"](L1Norm.png)  
  
This leads to  
!["Lasso MSE"](Lasso.png)  
  
Note:
* Regularization term is often multiplied by 2 (because of derivative of MSE that leads to leading 2)
* Division by m/size of dataset often done.
* Combination of both also possible.

However, there is no fixed rule

In [12]:
# Code just as above, only with penalty='l1' for Lasso regression

### Elastic Net Regression
Combination of Ridge Regression and Lasso Regression  
![Elastic Net Regression](ElasticNet.png)

In [14]:
from sklearn.linear_model import ElasticNet

elastic_net = ElasticNet(alpha=0.1, l1_ratio=0.5, random_state=42)
# l1_ratio: ratio of l1 and l2 regularization terms
elastic_net.fit(X, y)

(array([3.55886814]), array([0.81485127]))

### Comparison

Ridge Regression:
* Good default
* When expected that (almost) all features relevant or cross-correlation between features

--> Stabilizes model by reducing weight of coefficients, but will not set them to zero  
  
Lasso Regression:
* When expected that only few features are relevant

--> Can set weights of coefficients to zero  
  
Elastic Net:
* Especially useful when more features than observations or when features correlated
* Preferred over Lasso because Lasso might be erratic in cases above